# Attack Password with Correlation Power Analysis IV.1 (CPA)

In [ ]:
%run '../util/Metadata.ipynb'
print_metadata()

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Improving-the-code" data-toc-modified-id="Improving-the-code-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Improving the code</a></span></li><li><span><a href="#Basic-Setup" data-toc-modified-id="Basic-Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Basic Setup</a></span></li><li><span><a href="#Helper-Functions-for-Password-Attack" data-toc-modified-id="Helper-Functions-for-Password-Attack-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Helper Functions for Password Attack</a></span></li><li><span><a href="#New-Code---Old-results" data-toc-modified-id="New-Code---Old-results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>New Code - Old results</a></span></li><li><span><a href="#Disconnect" data-toc-modified-id="Disconnect-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Disconnect</a></span></li></ul></div>

In this example we want to discuss possibilities to fight against a CPA attack.

## Improving the code

Let's first recap the password checking loop from `advanced-passwdcheck-xor`:
```c
passbad = 0;
for(uint8_t i = 0; i < sizeof(correct_passwd); i++){
    passbad |= correct_passwd[i] ^ passwd[i];
}
```

We revealed in the last example that aboves XOR generates a collision between known and determinable input data and secret data which shall be revealed by an attack.

So, how to lower this correlation?

## Basic Setup

Define Variables

In [ ]:
%run "../util/Init.ipynb"

Build target and upload

In [ ]:
TARGET = 'simpleserial-passwordcheck'
%store TARGET
%run "$HELPERSCRIPTS/Prepare.ipynb"

Import helper functions

In [ ]:
%run "$HELPERSCRIPTS/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 500

## Helper Functions for Password Attack

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool, Label

output_notebook()

In [ ]:
import warnings
import random
import tqdm
import numpy as np

password_length = 8
"""Number of bytes of password"""

random_length = 32
"""Number of bytes of random input"""

def capture(command, data):
    scope.arm()
    target.simpleserial_write(command, data)

    ret = scope.capture()

    i = 0
    while not target.is_done():
        i += 1
        time.sleep(0.05)
        if i > 100:
            warnings.warn("Target did not finish operation")
            return None

    if ret:
        warnings.warn("Timeout happened during capture")
        return None

    return scope.get_last_trace()

def target_set_random(random_start=256):
    rand = bytes([random.randint(0, random_start)]) + bytes(random.choices(range(0, 256), k=random_length - 1))
    target.simpleserial_write('r', rand)
    return rand

def target_set_password(password):
    target.simpleserial_write('p', password)
    return target.simpleserial_read('r', password_length)

def target_check_password(command, password):
    target.simpleserial_write(command, password)
    return bytes(target.simpleserial_read('r', 1))[0] == 0

def capture_random(command, size=500, random_start=password_length, pass_guess=None):
    """Collect size number of password attempts with fully random random data."""
    traces = []
    textins = []
    rands = []
    for _ in tqdm.tqdm_notebook(range(size)):
        rands.append(target_set_random(random_start=random_start))
        pass_guess = bytes(random.choices(range(0, 256), k=password_length))
        traces.append(capture(command, pass_guess))
        textins.append(pass_guess)
    return np.array(traces), textins, rands

In [ ]:
import numpy as np
import warnings

warnings.simplefilter('ignore')

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    return HW[n]

hw_vec = np.vectorize(hw)

def pearson(x: np.array, y: np.array):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    return sum((x - x_mean) * (y - y_mean)) / np.sqrt(sum((x - x_mean) ** 2) * sum((y - y_mean) ** 2))

def pearson_helper_traces(traces):
    return traces_diff, traces_squared

def pearson_pointwise(traces, intermediates):
    n = len(intermediates)
    d_traces = traces - np.einsum('ij->j', traces, dtype='float64', optimize='optimal') / np.double(n)
    d_intermediates = intermediates - np.einsum('i->', intermediates, dtype='float64', optimize='optimal') / np.double(n)
    
    tmp = np.einsum('ij,ij->j', d_traces, d_traces, optimize='optimal')
    tmp *= np.einsum('i,i->', d_intermediates, d_intermediates, optimize='optimal')

    return np.dot(d_intermediates, d_traces) / np.sqrt(tmp)

def pearson_pointwise_multi(traces, intermediates):
    (n, t) = traces.shape
    (_, m) = intermediates.shape

    d_traces = traces - np.einsum('nt->t', traces, dtype='float64', optimize='optimal') / np.double(n)
    d_intermediates = intermediates - np.einsum('nm->m', intermediates, dtype='float64', optimize='optimal') / np.double(n)
    
    tmp1 = np.einsum('nm,nm->m', d_intermediates, d_intermediates, optimize='optimal')
    tmp2 = np.einsum('nt,nt->t', d_traces, d_traces, optimize='optimal')
    tmp = np.einsum('m,t->mt', tmp1, tmp2, optimize='optimal')
    denominator = np.sqrt(tmp)
    numerator = np.einsum('nm,nt->mt', d_intermediates, d_traces, optimize='optimal')

    return np.nan_to_num(numerator / denominator)

In [ ]:
import math
import bokeh.palettes
import bokeh.transform
from bokeh.models import ColumnDataSource

def plot_correlation(correlations, color_palette=bokeh.palettes.Oranges6, **kw):
    kw['height'] = kw.get('height', 300)
    kw['y_range'] = kw.get('y_range', (-1, 1))
    p = figure(sizing_mode='stretch_width', **kw)
    p.vbar(
        x='points',
        top='corr',
        width=1,
        source=ColumnDataSource(data=dict(
            points=range(len(correlations)),
            corr=correlations,
            abscorr=abs(correlations),
        )),
        color=bokeh.transform.linear_cmap(
            field_name='abscorr', 
            palette=color_palette,
            low=1,
            high=0,
        ),
    )
    return p

def plot_correlation_vs_traces(
    traces,
    textins,
    password_index=0,
    trylist='abcdefghijklmnopqrstuvwxyz0123456789',
    plotpoints=500,
):
    # Compute data
    plotpoints = min(plotpoints, len(traces))
    data = np.zeros((len(trylist), plotpoints))
    intermediates = np.array([[hw(attempt[password_index] ^ ord(guess)) for guess in trylist] for attempt in textins])
    for i in range(0, plotpoints):
        j = math.ceil(i / plotpoints * len(traces))
        data[:, i] = np.max(np.abs(pearson_pointwise_multi(traces[:j, :], intermediates[:j, :])), axis=1)

    # Create figure
    p = figure(sizing_mode='stretch_width')
    for color, guess, corrs in zip(len(data) * bokeh.palettes.Category20_20, trylist, data):
        p.line(x=range(2, len(traces), math.ceil(len(traces) / plotpoints)), y=corrs[2:], legend=guess, color=color)
    return p

## New Code - Old results

In [ ]:
password = b'ifx2019a'
target_set_password(password)
target_check_password('2', password)

In [ ]:
command = '1'

traces, textins, rands = capture_random(command=command, size=200)
correlations = pearson_pointwise(traces, np.array([hw(textin[0] ^ ord('i')) for textin in textins]))
print('max(correlations) = ', max(abs(correlations)))
show(plot_correlation(correlations))
show(plot_correlation_vs_traces(traces, textins))

In [ ]:
command = '2'

traces, textins, rands = capture_random(command=command, size=1000, random_start=1)
correlations = pearson_pointwise(traces, [hw(textin[0] ^ ord('i')) for textin in textins])
print('max(correlations) = ', max(abs(correlations)))
show(plot_correlation(correlations))
show(plot_correlation_vs_traces(traces, textins))

In [ ]:
command = '2'

traces, textins, rands = capture_random(command=command, size=10000, random_start=2)
correlations = pearson_pointwise(traces, [hw(textin[0] ^ ord('i')) for textin in textins])
print('max(correlations) = ', max(abs(correlations)))
show(plot_correlation(correlations))
show(plot_correlation_vs_traces(traces, textins))

In [ ]:
command = '2'

traces, textins, rands = capture_random(command=command, size=10000, random_start=3)
correlations = pearson_pointwise(traces, [hw(textin[0] ^ ord('i')) for textin in textins])
print('max(correlations) = ', max(abs(correlations)))
show(plot_correlation(correlations))
show(plot_correlation_vs_traces(traces, textins))

In [ ]:
command = '2'

traces, textins, rands = capture_random(command=command, size=30000)
correlations = pearson_pointwise(traces, [hw(textin[0] ^ ord('i')) for textin in textins])
print('max(correlations) = ', max(abs(correlations)))
show(plot_correlation(correlations))
show(plot_correlation_vs_traces(traces, textins))

## Disconnect

In [ ]:
scope.dis()
target.dis()